In [1]:
# Data Diri
"""
Tria Agusti Khoirun Nisa'
nisaagustin6@gmail.com
Watesprojo, Kemla, Mojokerto
Proyek Pertama : Membuat Model NLP dengan TensorFlow
"""

"\nTria Agusti Khoirun Nisa'\nnisaagustin6@gmail.com\nWatesprojo, Kemla, Mojokerto\nProyek Pertama : Membuat Model NLP dengan TensorFlow\n"

Mengimpor library yang dibutuhkan 


In [2]:
import pandas as pd
import tensorflow as tf
import csv
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Membaca dataset yang sudah diupload

In [3]:
df = pd.read_csv('spam.csv')
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


Melihat informasi yang terkandung di dalam dataset menggunakan fungsi df.info()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


Karena dataset merupakan data kategorikal, maka perlu melakukan proses one-hot-encoding

In [5]:
category = pd.get_dummies(df.Category)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='Category')
df_baru

,Message,ham,spam
0,"Go until jurong point, crazy.. Available only ...",1,0
1,Ok lar... Joking wif u oni...,1,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,0,1
3,U dun say so early hor... U c already then say...,1,0
4,"Nah I don't think he goes to usf, he lives aro...",1,0
...,...,...,...
5567,This is the 2nd time we have tried 2 contact u...,0,1
5568,Will ü b going to esplanade fr home?,1,0
5569,"Pity, * was in mood for that. So...any other s...",1,0
5570,The guy did some bitching but I acted like i'd...,1,0


Menghapus teks dan tanda baca yang tidak diperlukan dalam dataset

In [6]:
STOPWORDS = set(stopwords.words('english'))
articles = []
labels = []

with open("spam.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        article = row[1]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)

Mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values

In [7]:
message = df_baru['Message'].values
label = df_baru[['ham', 'spam']].values

Membagi dataset untuk data training dan data testing dengan validation set 20%

In [8]:
from sklearn.model_selection import train_test_split
message_train, message_test, label_train, label_test = train_test_split(message, label, test_size=0.2)

Mengubah setiap kata pada dataset ke dalam bilangan numerik dengan fungsi Tokenizer. Kemudian, membuat konversi untuk setiap sampel menjadi sequence

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(message_train) 
tokenizer.fit_on_texts(message_test)
 
sekuens_train = tokenizer.texts_to_sequences(message_train)
sekuens_test = tokenizer.texts_to_sequences(message_test)

padded_train = pad_sequences(sekuens_train,
                            padding='post',
                            maxlen=1500,
                            truncating='post')
padded_test = pad_sequences(sekuens_test,
                            padding='post',
                            maxlen=1500,
                            truncating='post')

Menerapkan Embedding dengan menggunakan dimensi embbeding sebesar 16, serta dimensi iinput sebesar nilai num_words pada objek tokonizer. 
Kemudian, memanggil fungsi compile dan menentukan optimizer serta loss function yang akan dipakai oleh model

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          80000     
                                                                 
 lstm (LSTM)                 (None, 64)                20736     
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 117,442
Trainable params: 117,442
Non-trainable params: 0
_________________________________________________________________
None


Menerapkan callback pada model 

In [11]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.90):
      print("\nAkurasi telah mencapai >90%!")
      self.model.stop_training = True
callbacks = myCallback()

Melatih model dengan memanggil fungsi fit()

In [12]:
num_epochs = 15
history = model.fit(padded_train, label_train, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/15
140/140 - 13s - loss: 0.4331 - accuracy: 0.8618 - val_loss: 0.3620 - val_accuracy: 0.8825 - 13s/epoch - 94ms/step
Epoch 2/15
140/140 - 6s - loss: 0.4050 - accuracy: 0.8618 - val_loss: 0.3619 - val_accuracy: 0.8825 - 6s/epoch - 43ms/step
Epoch 3/15
140/140 - 6s - loss: 0.4055 - accuracy: 0.8618 - val_loss: 0.3656 - val_accuracy: 0.8825 - 6s/epoch - 45ms/step
Epoch 4/15
140/140 - 6s - loss: 0.4038 - accuracy: 0.8618 - val_loss: 0.3620 - val_accuracy: 0.8825 - 6s/epoch - 43ms/step
Epoch 5/15
140/140 - 6s - loss: 0.4047 - accuracy: 0.8618 - val_loss: 0.3622 - val_accuracy: 0.8825 - 6s/epoch - 45ms/step
Epoch 6/15
140/140 - 6s - loss: 0.4045 - accuracy: 0.8618 - val_loss: 0.3620 - val_accuracy: 0.8825 - 6s/epoch - 43ms/step
Epoch 7/15
140/140 - 6s - loss: 0.4036 - accuracy: 0.8618 - val_loss: 0.3655 - val_accuracy: 0.8825 - 6s/epoch - 44ms/step
Epoch 8/15
140/140 - 6s - loss: 0.4037 - accuracy: 0.8618 - val_loss: 0.3634 - val_accuracy: 0.8825 - 6s/epoch - 43ms/step
Epoch 9/15
140